Script below takes health indicators from https://data.hrsa.gov/maps/mchb and specifies the important columns, aggregating all sheets into one excel file.  Download each health indicator per state and get all the county data for them. Note: only works for specific columns in "columns_to_keep" variable- can subtract from this, but I don't recommend adding without expecting to make some adjustments for different sized data.

In [56]:
import pandas as pd
import os

# Path to your folder (could not get relative path to work for whatever reason *eyeroll*
folder_path = "/Users/eclark/Python/FinalProject/Data/2017_2019" 

# Initialize a master dataframe
master_df = None

# Specify columns to keep
columns_to_keep = ["County FIPS Code", "HRSA Region", "State", "County", 
                   "(#) Women 15-44 Years of Age by Race - Asian", 
                   "(%) Women 15-44 Years of Age by Race - Asian", 
                   "NCHS Urban-Rural Classification", 
                   "Households with Low Income (< 200% of the Poverty Level) (%)", 
                   "(#) Women 15-44 Years of Age by Race - More than one race", 
                   "(%) Women 15-44 Years of Age by Race - More than one race", 
                   "OB-GYN Provider Rate (per 100,000)", 
                   "(#) Women 15-44 Years of Age by Race - White", 
                   "(%) Women 15-44 Years of Age by Race - White",
                   "Uninsured Women 18-49 Years of Age (%)", 
                   "Prenatal Care in the 1st Trimester (%) 2017-2019", 
                   "(#) Women 15-44 Years of Age by Race - Black or African American", 
                   "(%) Women 15-44 Years of Age by Race - Black or African American",
                   "(#) Women 15-44 Years of Age by Race - American Indian or Alaska Native",
                   "(%) Women 15-44 Years of Age by Race - American Indian or Alaska Native",
                   "Obesity - Pre-pregnancy (%) 2017-2019",
                   "(#) Women 15-44 Years of Age by Ethnicity - Hispanic or Latina",
                   "(%) Women 15-44 Years of Age by Ethnicity - Hispanic or Latina",
                   "(#) Women 15-44 Years of Age by Race - Native Hawaiian or Other Pacific Islander",
                   "(%) Women 15-44 Years of Age by Race - Native Hawaiian or Other Pacific Islander",
                   "Diabetes - Pre-pregnancy (%) 2017-2019",
                   "Hypertension - Pre-pregnancy (%) 2017-2019",
                   "(#) Women 15-44 Years of Age by Ethnicity - Not Hispanic or Latina",
                   "(%) Women 15-44 Years of Age by Ethnicity - Not Hispanic or Latina",
                   "Cesarean Delivery - All (%) 2017-2019",
                   "Birth Rate (per 1,000)",
                   "Year Range"]

# Loop through files in the folder
for file in os.listdir(folder_path):
    if file.endswith('.xlsx') and not file.startswith('~$'):  # Exclude temp files
        file_path = os.path.join(folder_path, file)
        
        try:
            # Load the data without skipping rows
            df = pd.read_excel(file_path, engine='openpyxl')
            
            # Ensure 'State' and 'County' columns exist before merging
            if 'State' not in df.columns or 'County' not in df.columns:
                print(f"Skipping file {file}: Missing 'State' or 'County' column.")
                continue

            # Keep only the desired columns for this file
            df = df.filter(columns_to_keep, axis=1)
            
            # Merge datasets based on 'State' and 'County' with unique suffixes for overlaps
            if master_df is None:
                master_df = df  # Initialize the master dataframe
            else:
                master_df = pd.merge(
                    master_df, df, 
                    on=['State', 'County'], 
                    how='outer', 
                    suffixes=('', f'_{file[:5]}')  # Add file-based suffix for duplicate columns
                )
        
        except Exception as e:
            print(f"Error processing file {file}: {e}")

# Remove duplicate columns after merging
if master_df is not None:
    # Keep only the first occurrence of duplicate columns based on the original column names
    master_df = master_df.loc[:, ~master_df.columns.duplicated()]

    # Keep only the desired columns in the final dataset
    master_df = master_df.filter(columns_to_keep, axis=1)
    
    # Save the cleaned, merged dataframe
    master_df.to_excel('merged_health_indicators.xlsx', index=False)
    print("Merged dataset saved as 'merged_health_indicators.xlsx'.")
else:
    print("No valid Excel files were processed.")

Merged dataset saved as 'merged_health_indicators.xlsx'.
